In [1]:
# ============================================
# OLIVE PROCESS-BASED MODEL (ENGLISH VERSION)
# Reference: Moriondo et al. (2019) / Villalobos et al. (2013)
# Configuration: Super-Intensive Arbequina (Chile)
# ============================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


columns = ["T2M_MIN", "T2M_MAX", "PRECTOTCORR", "ALLSKY_SFC_SW_DWN", "RH2M", "WS2M"]

Omega = np.load("../genai/scenarios.npy")
df = pd.DataFrame(Omega[0], columns=columns)


print(df.columns)
fixed_window = pd.read_csv('../genai/fixed_window.csv', index_col=0)
print(fixed_window.columns)
fixed_window.columns = columns
print(fixed_window.columns)
df = pd.concat([fixed_window, df])
print(df.columns)
try:
    
    #df['T2M'] = (df["T2M_MIN"] + df["T2M_MAX"])/2 
    print(df.columns)
    #df.rename(columns={"ALLSKY_SFC_SW_DWN":"RS", "CLRSKY_SFC_SW_DWN": "RSO",
    #            "PRECTOTCORR": "PRECIPITATION"}, inplace=True)
    # CLRSKY_SFC_SW_DWN?
    #df['RSO'] = df['RS']
    #print(df.columns)
    # Merge with real data and fix scenario generation according tactical planning  
    df_season = df#.loc['2022-11-01':'2023-04-29'].copy()
    if len(df_season) < 180:
        print(f"Warning: Season data is short ({len(df_season)} days). Using available data.")
    else:
        df_season = df_season.iloc[:180]
    print(f"Loaded {len(df_season)} days of weather data for simulation.")
except FileNotFoundError:
    print("="*50)
    print("ERROR: Data file not found. Please re-run Cell 4 to download the data.")
    print("="*50)

Index(['T2M_MIN', 'T2M_MAX', 'PRECTOTCORR', 'ALLSKY_SFC_SW_DWN', 'RH2M',
       'WS2M'],
      dtype='object')
Index(['0', '1', '2', '3', '4', '5'], dtype='object')
Index(['T2M_MIN', 'T2M_MAX', 'PRECTOTCORR', 'ALLSKY_SFC_SW_DWN', 'RH2M',
       'WS2M'],
      dtype='object')
Index(['T2M_MIN', 'T2M_MAX', 'PRECTOTCORR', 'ALLSKY_SFC_SW_DWN', 'RH2M',
       'WS2M'],
      dtype='object')
Index(['T2M_MIN', 'T2M_MAX', 'PRECTOTCORR', 'ALLSKY_SFC_SW_DWN', 'RH2M',
       'WS2M'],
      dtype='object')
Loaded 180 days of weather data for simulation.


In [2]:
df_season.reset_index(inplace=True)


In [3]:
print(df)

from datetime import date, timedelta

start_date = date(2024, 8, 30)
# Create a list of 180 days, starting from the day after start_date
date_list = [start_date + timedelta(days=x) for x in range(1, 181)]

df['DATE'] = date_list

df.DATE = pd.to_datetime(df.DATE)
df.set_index('DATE', inplace=True)

     T2M_MIN    T2M_MAX  PRECTOTCORR  ALLSKY_SFC_SW_DWN       RH2M      WS2M
0   6.830000  18.260000     0.000000           6.540000  82.480000  1.130000
1   6.200000  15.340000     0.000000           6.370000  84.810000  1.240000
2   4.590000  16.790000     0.000000          15.850000  74.550000  2.050000
3   0.970000  23.150000     0.000000          18.940000  63.160000  1.980000
4   4.630000  21.940000     0.000000          19.380000  66.540000  1.190000
..       ...        ...          ...                ...        ...       ...
55  8.750850  13.817754     2.698063           7.132063  77.296646  1.693531
56  5.466936  18.642084     0.740222          23.911379  72.229591  1.372966
57  4.688178  20.801809     0.128090          29.092003  70.570923  1.582005
58  6.155434  23.774721    -0.291742          26.166977  66.427025  1.409660
59  8.226084  26.256895    -0.175512          25.574631  67.752052  1.665095

[180 rows x 6 columns]


## Yearly

In [ ]:
# ==========================================================
# OLIVE ORCHARD MODEL (Chile – Terramater / Olivares de Peteroa)
# Multi-season version (Sep 1 -> May 31)
# - Runs all available seasons in the selected date window
# - Builds a season synthesis table
# - Plots multi-season mean ± 95% CI (season-day aligned)
# ==========================================================

VERBOSE = True

# ==========================================================
# 2) PRE-PROCESS WEATHER DRIVERS
# ==========================================================
df["TAVG"] = (df["T2M_MAX"] + df["T2M_MIN"]) / 2.0

# Radiation unit auto-detection
allsky_mean = float(df["ALLSKY_SFC_SW_DWN"].mean())
if allsky_mean < 12.0:
    rad_factor = 3.6  # kWh/m2/day -> MJ/m2/day
    if VERBOSE:
        print("[INFO] Radiation units likely kWh/m²/day; converting to MJ/m²/day (×3.6).")
else:
    rad_factor = 1.0
    if VERBOSE:
        print("[INFO] Radiation units likely MJ/m²/day; keeping (×1.0).")

df["SRAD_MJ"] = df["ALLSKY_SFC_SW_DWN"] * rad_factor          # energy driver
df["PAR_MJ"]  = df["SRAD_MJ"] * 0.48                          # proxy PAR for growth

# VPD (kPa) from TAVG and RH2M
es = 0.6108 * np.exp(17.27 * df["TAVG"] / (df["TAVG"] + 237.3))
ea = es * (df["RH2M"] / 100.0)
df["VPD_kPa"] = np.maximum(0.1, es - ea)

# Rain (mm)
df["RAIN_mm"] = df["PRECTOTCORR"]

if VERBOSE:
    print(f"[INFO] Mean ALLSKY={allsky_mean:.2f} | Mean SRAD_MJ={df['SRAD_MJ'].mean():.2f} | Mean PAR_MJ={df['PAR_MJ'].mean():.2f}")


# ==========================================================
# 3) PARAMETERS (site-calibrated values should be tuned with field records)
# ==========================================================
params = dict(
    # orchard structure
    PlantD=1600.0,                     # trees/ha
    PlantA=10000.0 / 1600.0,            # m2/tree
    LAD=2.0,                            # m2/m3

    # phenology (daily approximation)
    chill_a=0.01,
    chill_b=-0.50,
    chill_c=7.20,
    Ccrit=185.0, #paper 185 here 350

    forcing_d=0.50,
    forcing_e=9.00,
    FcritFlo=450.0,

    # growth
    RUE_ol=0.98,                        # g DM / MJ
    SLA_ol=5.2,                         # m2 / kg
    HI_pot_base=0.35,
    harvest_doy=120,                    # calibrate with farm records
    PClf_pot_base=0.18,

    # canopy light
    Ck1=0.52, Ck2=0.000788, Ck3=0.76, Ck4=1.25,

    # soil profile
    TTSW1=70.0,                         # mm
    TTSW2=110.0,                        # mm
    Initial_Saturation=0.90,
    root_frac_layer1=40.0 / 120.0,
    root_frac_layer2=80.0 / 120.0,

    # stress response (logistic-shaped)
    RelTr_a=6.17,
    RelTr_b=13.45,
    RelLAI_a=78.24,
    RelLAI_b=21.42,

    # transpiration efficiency
    TE_coeff=4.0,                       # (paper uses Pa; here used consistently as a coefficient)

    # irrigation management
    Irrigation_Threshold=0.20,
    Irrigation_Efficiency=0.95,

    # soil evaporation
    SALB=0.20,
    gamma_kpa=0.68,

    # HI penalties around anthesis (averaged window)
    FTSWo=0.40,
    FTSWm=0.05,
    TMAXo=30.0,
    TMAXm=40.0,
    anthesis_window_days=7,

    # conversion
    fresh_factor=2.2,
)

# initial state
params["LAI_ini"] = 3.5


# ==========================================================
# 4) CORE FUNCTIONS (no equation mapping in comments, per your request)
# ==========================================================
def unichill(Tavg, a, b, c):
    z = a * (Tavg - c) ** 2 + b * (Tavg - c)
    return 1.0 / (1.0 + np.exp(z))

def forcing_logistic(Tavg, d, e):
    z = -d * (Tavg - e)
    return 1.0 / (1.0 + np.exp(z))

def k_prime(PlantD, LAD, p):
    return p["Ck1"] + p["Ck2"] * PlantD - p["Ck3"] * np.exp(-p["Ck4"] * LAD)

def canopy_volume_from_lai(LAI, LAD):
    return LAI / max(1e-9, LAD)

def intercepted_fraction(kp, v):
    return 1.0 - np.exp(-kp * v)

def dm_potential(Int, RAD_MJ, RUE):
    return Int * RAD_MJ * RUE  # g/m2/day

def te_coeff(Kd, VPD_kpa):
    return Kd / max(1e-6, VPD_kpa)

def tr_potential(DM_pot_gm2, TE):
    return DM_pot_gm2 / max(1e-9, TE)

def rel_factor(FTSW, a, b):
    return 1.0 / (1.0 + a * np.exp(-b * FTSW))

def rel_te(RelTr):
    return -0.74 * RelTr + 1.74

def dm_actual(DM_pot, RelTr, RelTE):
    return DM_pot * RelTr * RelTE

def lai_increment(DM_pot_gm2, PClf, SLA_m2_per_kg):
    dm_leaf_kgm2 = (DM_pot_gm2 * PClf) / 1000.0
    return dm_leaf_kgm2 * SLA_m2_per_kg

def lai_senescence(doy, YLAI_y_minus_2, DOY_ini=250, DOY_end=330):
    if YLAI_y_minus_2 <= 0:
        return 0.0
    if DOY_ini <= doy <= DOY_end:
        return YLAI_y_minus_2 / float(DOY_end - DOY_ini + 1)
    return 0.0

def delta_kpa_per_c(Tavg):
    es = 0.6108 * np.exp(17.27 * Tavg / (Tavg + 237.3))
    return 4098.0 * es / ((Tavg + 237.3) ** 2)

def sevp_potential(SRAD_MJ, SALB, INT_tot, Delta, gamma_kpa=0.68, lambda_mj_per_mm=2.45):
    net_rad = SRAD_MJ * (1.0 - SALB) * (1.0 - INT_tot)
    mm_equiv = net_rad / max(1e-9, lambda_mj_per_mm)
    return mm_equiv * (Delta / max(1e-9, (Delta + gamma_kpa)))

def sevp_actual(sevp_pot, DYSE):
    DYSE = max(1.0, float(DYSE))
    return sevp_pot * (np.sqrt(DYSE) - np.sqrt(max(0.0, DYSE - 1.0)))

def recharge_two_layers(ATSW1, ATSW2, Rain, Ir, TTSW1, TTSW2):
    ATSW1_new = ATSW1 + Rain + Ir
    excess = max(0.0, ATSW1_new - TTSW1)
    ATSW1_new = min(ATSW1_new, TTSW1)
    ATSW2_new = min(ATSW2 + excess, TTSW2)
    return ATSW1_new, ATSW2_new

def ftsw1(ATSW1, TTSW1):
    return ATSW1 / max(1e-9, TTSW1)

def ftsw_profile(ATSW1, ATSW2, TTSW1, TTSW2):
    return (ATSW1 + ATSW2) / max(1e-9, (TTSW1 + TTSW2))

def hi_water(HI_pot, FTSWant, FTSWo=0.40, FTSWm=0.05):
    if FTSWant >= FTSWo:
        return HI_pot
    if FTSWant <= FTSWm:
        return 0.0
    return HI_pot * (1.0 - (FTSWo - FTSWant) / max(1e-9, (FTSWo - FTSWm)))

def hi_heat(HI_pot, TMAXant, TMAXo=30.0, TMAXm=40.0):
    if TMAXant <= TMAXo:
        return HI_pot
    if TMAXant >= TMAXm:
        return 0.0
    return HI_pot * (1.0 - (TMAXant - TMAXo) / max(1e-9, (TMAXm - TMAXo)))

def update_next_season(HIa, HI_pot_base=0.35, PClf_pot_base=0.18):
    HI_pot_next = max(0.0, min(0.70, 0.70 - HIa))
    PClf_next = max(0.0, PClf_pot_base * (0.65 + HIa))
    return HI_pot_next, PClf_next

def abi_index(yields):
    ys = [y for y in yields if y is not None and not np.isnan(y)]
    n = len(ys)
    if n < 2:
        return np.nan
    terms = []
    for i in range(n - 1):
        denom = ys[i] + ys[i + 1]
        terms.append(abs(ys[i] - ys[i + 1]) / denom if denom > 0 else 0.0)
    return sum(terms) / (n - 1)


# ==========================================================
# 5) SIMULATION
# ==========================================================
LAI = params["LAI_ini"]
ATSW1 = params["TTSW1"] * params["Initial_Saturation"]
ATSW2 = params["TTSW2"] * params["Initial_Saturation"]
DYSE = 1

chill_cum = 0.0
forcing_cum = 0.0
dormancy_released = False
anthesis_occurred = False
anthesis_date = None

HI_pot = params["HI_pot_base"]
HIa = HI_pot
HIa_fixed = False
anthesis_FTSW_samples = []
anthesis_TMAX_samples = []

DM_cum = 0.0

year_state = dict(
    YLAI_prod_by_season={},
    HIa_by_season={},
    yield_by_season={},
)
YLAI_produced_this_season = 0.0

results = []

if VERBOSE:
    print(f"\n--- STARTING SIMULATION:  ---")
    print(f"Date window: {df.index.min().date()} to {df.index.max().date()}")
    print(f"Irrigation trigger: FTSW < {params['Irrigation_Threshold']:.2f}")
    print(f"Harvest DOY: {params['harvest_doy']} (calibrate if needed)\n")

for date, row in df.iterrows():
    year = int(date.year)
    doy = int(date.dayofyear)

    # Chile agronomic season-year
    season_year = year if date.month >= 9 else year - 1

    # Season reset at Sep 1
    if (date.month == 8) and (date.day == 31):
        prev_season = season_year - 1

        if prev_season not in year_state["YLAI_prod_by_season"]:
            year_state["YLAI_prod_by_season"][prev_season] = YLAI_produced_this_season
        YLAI_produced_this_season = 0.0

        prev_HIa = year_state["HIa_by_season"].get(prev_season, params["HI_pot_base"])
        HI_pot, PClf_next = update_next_season(
            prev_HIa,
            HI_pot_base=params["HI_pot_base"],
            PClf_pot_base=params["PClf_pot_base"]
        )
        params["PClf_pot_base"] = PClf_next

        chill_cum = 0.0
        forcing_cum = 0.0
        dormancy_released = False
        anthesis_occurred = False
        anthesis_date = None
        HIa = HI_pot
        HIa_fixed = False
        anthesis_FTSW_samples = []
        anthesis_TMAX_samples = []
        DM_cum = 0.0
        DYSE = 1

        if VERBOSE:
            print(f"--- SEASON {season_year}-{season_year+1} RESET (Sep 1) ---")
            print(f"    HI_pot={HI_pot:.3f} | PClf_pot={params['PClf_pot_base']:.3f}")

    Tavg = float(row["TAVG"])

    # Phenology (daily approximation, scaled by 24)
    if not dormancy_released:
        CU = unichill(Tavg, params["chill_a"], params["chill_b"], params["chill_c"])
        chill_cum += CU * 24.0
        if chill_cum >= params["Ccrit"]:
            dormancy_released = True
            if VERBOSE:
                print(f"[PHENO] {date.date()} dormancy released")
    elif not anthesis_occurred:
        FU = forcing_logistic(Tavg, params["forcing_d"], params["forcing_e"])
        forcing_cum += FU * 24.0
        if forcing_cum >= params["FcritFlo"]:
            anthesis_occurred = True
            anthesis_date = date
            if VERBOSE:
                print(f"[PHENO] {date.date()} anthesis reached")

    # Canopy and light interception
    kp = k_prime(params["PlantD"], params["LAD"], params)
    v  = canopy_volume_from_lai(LAI, params["LAD"])
    Int_OT = intercepted_fraction(kp, v)

    # Growth
    PAR_MJ = float(row["PAR_MJ"])
    DM_pot = dm_potential(Int_OT, PAR_MJ, params["RUE_ol"])

    # Irrigation + recharge
    total_capacity = params["TTSW1"] + params["TTSW2"]
    current_FTSW = ftsw_profile(ATSW1, ATSW2, params["TTSW1"], params["TTSW2"])

    irrigation_today = 0.0
    if (current_FTSW < params["Irrigation_Threshold"]) and (date.month in [9, 10, 11, 12, 1, 2, 3, 4]):
        deficit = total_capacity - (ATSW1 + ATSW2)
        irrigation_today = deficit / max(1e-9, params["Irrigation_Efficiency"])

    Rain = float(row["RAIN_mm"])
    Ir = float(irrigation_today)

    ATSW1, ATSW2 = recharge_two_layers(ATSW1, ATSW2, Rain, Ir, params["TTSW1"], params["TTSW2"])

    FTSW1_val = ftsw1(ATSW1, params["TTSW1"])
    FTSW_val  = ftsw_profile(ATSW1, ATSW2, params["TTSW1"], params["TTSW2"])

    # Stress and actual biomass
    RelTr_val  = rel_factor(FTSW_val, params["RelTr_a"], params["RelTr_b"])
    RelLAI_val = rel_factor(FTSW_val, params["RelLAI_a"], params["RelLAI_b"])
    RelTE_val  = rel_te(RelTr_val)
    DM_act     = dm_actual(DM_pot, RelTr_val, RelTE_val)

    # Transpiration (proxy)
    VPD_kPa = float(row["VPD_kPa"])
    TE_val  = te_coeff(params["TE_coeff"], VPD_kPa)
    Tr_pot  = tr_potential(DM_pot, TE_val)
    Tr_act  = Tr_pot * RelTr_val

    # Soil evaporation
    if (Rain + Ir) > 0.5:
        DYSE = 1
    else:
        DYSE += 1

    SRAD_MJ = float(row["SRAD_MJ"])
    Delta   = delta_kpa_per_c(Tavg)
    SEVP_pot = sevp_potential(SRAD_MJ, params["SALB"], Int_OT, Delta, params["gamma_kpa"])
    SEVP     = sevp_actual(SEVP_pot, DYSE)

    # Water uptake by layers
    Tr1 = Tr_act * params["root_frac_layer1"]
    Tr2 = Tr_act * params["root_frac_layer2"]

    ATSW1 -= (Tr1 + SEVP)
    if ATSW1 < 0:
        deficit = -ATSW1
        ATSW1 = 0.0
        ATSW2 = max(0.0, ATSW2 - deficit)

    ATSW2 = max(0.0, ATSW2 - Tr2)

    FTSW1_val = ftsw1(ATSW1, params["TTSW1"])
    FTSW_val  = ftsw_profile(ATSW1, ATSW2, params["TTSW1"], params["TTSW2"])

    # HI fixation around anthesis
    if anthesis_occurred and (not HIa_fixed):
        anthesis_FTSW_samples.append(FTSW_val)
        anthesis_TMAX_samples.append(float(row["T2M_MAX"]))

        if len(anthesis_FTSW_samples) >= params["anthesis_window_days"]:
            FTSWant = float(np.mean(anthesis_FTSW_samples))
            TMAXant = float(np.mean(anthesis_TMAX_samples))

            HIws = hi_water(HI_pot, FTSWant, params["FTSWo"], params["FTSWm"])
            HIhs = hi_heat(HI_pot,  TMAXant, params["TMAXo"], params["TMAXm"])

            # Combined penalty rule
            HIa = max(0.0, min(HI_pot, HIws + HIhs - HI_pot))
            HIa_fixed = True

            year_state["HIa_by_season"][season_year] = HIa

            if VERBOSE:
                print(f"[HI] {date.date()} HIa fixed | HIa={HIa:.3f}")

    # LAI dynamics
    PClf = params["PClf_pot_base"]
    LAI_inc_pot = lai_increment(DM_pot, PClf, params["SLA_ol"])
    ALAI_inc    = LAI_inc_pot * RelLAI_val

    if ALAI_inc > 0:
        YLAI_produced_this_season += ALAI_inc

    YLAI_y_minus_2 = year_state["YLAI_prod_by_season"].get(season_year - 2, 0.0)
    LAI_sen = lai_senescence(doy, YLAI_y_minus_2)

    LAI = max(0.1, LAI + ALAI_inc - LAI_sen)

    # Yield accumulation stop at harvest
    harvest_passed = (date.month <= 6) and (doy >= int(params["harvest_doy"]))
    if dormancy_released and (not harvest_passed):
        DM_cum += DM_act

    Yield_gm2 = (HIa if HIa_fixed else HI_pot) * DM_cum

    results.append(dict(
        DATE=date,
        SEASON_YEAR=season_year,
        DOY=doy,
        LAI=LAI,
        Int_OT=Int_OT,
        DM_pot=DM_pot,
        DM_act=DM_act,
        DM_cum=DM_cum,
        HI_pot=HI_pot,
        HIa=HIa if HIa_fixed else np.nan,
        Yield_gm2=Yield_gm2,
        ATSW1=ATSW1,
        ATSW2=ATSW2,
        FTSW1=FTSW1_val,
        FTSW=FTSW_val,
        RelTr=RelTr_val,
        RelLAI=RelLAI_val,
        RelTE=RelTE_val,
        TE=TE_val,
        Tr_pot=Tr_pot,
        Tr_act=Tr_act,
        SEVP_pot=SEVP_pot,
        SEVP=SEVP,
        DYSE=DYSE,
        Rain=Rain,
        Irrigation=Ir,
        VPD_kPa=VPD_kPa,
        dormancy_released=dormancy_released,
        anthesis_occurred=anthesis_occurred,
        harvest_passed=harvest_passed,
    ))

res_df = pd.DataFrame(results).set_index("DATE")


# ==========================================================
# 6) MULTI-SEASON SYNTHESIS + ABI
# ==========================================================
def season_windows_from_index(idx, start_md=(9,1), end_md=(5,31), require_complete=True):
    idx = pd.DatetimeIndex(idx)
    min_y = idx.min().year
    max_y = idx.max().year
    windows = []
    for y in range(min_y, max_y + 1):
        start = pd.Timestamp(y, start_md[0], start_md[1])
        end   = pd.Timestamp(y + 1, end_md[0], end_md[1])
        if require_complete and (start < idx.min() or end > idx.max()):
            continue
        if (end >= idx.min()) and (start <= idx.max()):
            windows.append((y, start, end))
    return windows

def first_true_date(s_bool):
    if s_bool is None or len(s_bool) == 0:
        return pd.NaT
    s = s_bool.astype(int)
    d = s.diff().fillna(0)
    hits = d[d == 1]
    return hits.index.min() if len(hits) else pd.NaT

def mean_ci(x, conf=0.95):
    x = np.array([v for v in x if v is not None and not np.isnan(v)], dtype=float)
    n = len(x)
    if n == 0:
        return np.nan, np.nan, np.nan, n
    mu = float(np.mean(x))
    if n == 1:
        return mu, np.nan, np.nan, n
    sd = float(np.std(x, ddof=1))
    se = sd / np.sqrt(n)
    alpha = 1.0 - conf
    try:
        from scipy.stats import t
        crit = float(t.ppf(1.0 - alpha/2.0, df=n-1))
    except Exception:
        crit = 1.96
    lo = mu - crit * se
    hi = mu + crit * se
    return mu, lo, hi, n

windows = season_windows_from_index(res_df.index, (9,1), (5,31), require_complete=True)
if VERBOSE:
    print("\n[INFO] Complete seasons (Sep->May):", [f"{y}-{y+1}" for y,_,_ in windows])

season_rows = []
for y, start, end in windows:
    s = res_df.loc[start:end].copy()
    w = df.loc[start:end].copy()

    y_gm2 = float(s["Yield_gm2"].max())
    y_dm_kg_ha = y_gm2 * 10.0
    y_fresh_kg_ha = y_dm_kg_ha * params["fresh_factor"]
    irrig_mm = float(s["Irrigation"].sum())

    dorm_date = first_true_date(s["dormancy_released"])
    anth_date = first_true_date(s["anthesis_occurred"])

    hi_series = s["HIa"].dropna()
    hi_a = float(hi_series.iloc[0]) if len(hi_series) else np.nan




    season_rows.append({
        "Season": f"{y}-{y+1}",
        "Start": start.date(),
        "End": end.date(),
        "DormancyRelease": dorm_date.date() if pd.notna(dorm_date) else None,
        "Anthesis": anth_date.date() if pd.notna(anth_date) else None,
        "HIa": hi_a,
        "Yield_DM_kg_ha": y_dm_kg_ha,
        "Yield_Fresh_kg_ha": y_fresh_kg_ha,
        "Irrigation_mm": irrig_mm,
        "LAI_max": float(s["LAI"].max()),
        "FTSW_mean": float(s["FTSW"].mean()),
        "VPD_mean_kPa": float(s["VPD_kPa"].mean()),
        "Tmin_mean_C": float(w["T2M_MIN"].mean()) if "T2M_MIN" in w.columns else np.nan,
        "Tmax_mean_C": float(w["T2M_MAX"].mean()) if "T2M_MAX" in w.columns else np.nan,
    })

season_summary = pd.DataFrame(season_rows)

abi = abi_index(season_summary["Yield_DM_kg_ha"].values.tolist()) if len(season_summary) else np.nan
mu_y, lo_y, hi_y, n_y = mean_ci(season_summary["Yield_DM_kg_ha"].values, conf=0.95)
mu_i, lo_i, hi_i, n_i = mean_ci(season_summary["Irrigation_mm"].values, conf=0.95)

print("\n" + "="*80)
print("SEASON SYNTHESIS TABLE (Chile Sep->May)")
print("="*80)
print(season_summary.round(3))

print("\n" + "-"*80)
print(f"Yield_DM (kg/ha): mean={mu_y:.0f} | 95% CI [{lo_y:.0f}, {hi_y:.0f}] | n={n_y}")
print(f"Irrigation (mm):  mean={mu_i:.0f} | 95% CI [{lo_i:.0f}, {hi_i:.0f}] | n={n_i}")
print(f"ABI:              {abi:.3f}")
print("-"*80)

season_summary.to_csv(OUT_TABLE_PATH)
print(f"[INFO] Synthesis table saved to: {OUT_TABLE_PATH}")


# ==========================================================
# 7) MULTI-SEASON CI PLOTS (season-day aligned)
# ==========================================================
def align_by_season_day(df_in, windows, col):
    max_len = max(int((end - start).days) + 1 for _, start, end in windows)
    mat = np.full((len(windows), max_len), np.nan, dtype=float)
    for i, (y, start, end) in enumerate(windows):
        ss = df_in.loc[start:end, col].astype(float).values
        mat[i, :len(ss)] = ss
    return mat

def ci_band_from_matrix(mat):
    mean = np.nanmean(mat, axis=0)
    std  = np.nanstd(mat, axis=0, ddof=1)
    n    = np.sum(~np.isnan(mat), axis=0)
    se   = np.where(n > 0, std / np.sqrt(np.maximum(n, 1)), np.nan)
    crit = 1.96  # normal approx for visual stability
    lo = mean - crit * se
    hi = mean + crit * se
    return mean, lo, hi, n

def plot_multiseason_ci(df_weather, res_df, windows, params, save_figs=True, fig1_path=FIG1_PATH, fig2_path=FIG2_PATH):
    if len(windows) == 0:
        print("[WARN] No complete seasons to plot.")
        return

    dm_pot_mat = align_by_season_day(res_df, windows, "DM_pot")
    dm_act_mat = align_by_season_day(res_df, windows, "DM_act")
    ftsw_mat   = align_by_season_day(res_df, windows, "FTSW")
    lai_mat    = align_by_season_day(res_df, windows, "LAI")
    yld_mat    = align_by_season_day(res_df, windows, "Yield_gm2")
    irr_mat    = align_by_season_day(res_df, windows, "Irrigation")
    vpd_mat    = align_by_season_day(res_df, windows, "VPD_kPa")
    tr_mat     = align_by_season_day(res_df, windows, "Tr_act")
    reltr_mat  = align_by_season_day(res_df, windows, "RelTr")

    tmin_mat   = align_by_season_day(df_weather, windows, "T2M_MIN")
    tmax_mat   = align_by_season_day(df_weather, windows, "T2M_MAX")

    dm_pot_mu, dm_pot_lo, dm_pot_hi, _ = ci_band_from_matrix(dm_pot_mat)
    dm_act_mu, dm_act_lo, dm_act_hi, _ = ci_band_from_matrix(dm_act_mat)
    ftsw_mu,   ftsw_lo,   ftsw_hi,   _  = ci_band_from_matrix(ftsw_mat)
    lai_mu,    lai_lo,    lai_hi,    _  = ci_band_from_matrix(lai_mat)
    yld_mu,    yld_lo,    yld_hi,    _  = ci_band_from_matrix(yld_mat)

    irr_cum_mat = np.nancumsum(irr_mat, axis=1)
    irr_mu, irr_lo, irr_hi, _ = ci_band_from_matrix(irr_cum_mat)

    tmin_mu, tmin_lo, tmin_hi, _ = ci_band_from_matrix(tmin_mat)
    tmax_mu, tmax_lo, tmax_hi, _ = ci_band_from_matrix(tmax_mat)
    vpd_mu,  vpd_lo,  vpd_hi,  _ = ci_band_from_matrix(vpd_mat)
    tr_mu,   tr_lo,   tr_hi,   _ = ci_band_from_matrix(tr_mat)
    reltr_mu,reltr_lo,reltr_hi,_ = ci_band_from_matrix(reltr_mat)

    x = np.arange(len(dm_act_mu))

    # Figure 1
    fig1, axes = plt.subplots(2, 2, figsize=(16, 10))
    # fig1.suptitle("Figure 1 (Multi-season): Production & Water Balance (Mean ± 95% CI)",
    #               fontsize=16, fontweight="bold")

    ax = axes[0, 0]
    ax.plot(x, dm_pot_mu, lw=1.8, label="Potential biomass (DM)")
    ax.fill_between(x, dm_pot_lo, dm_pot_hi, alpha=0.15)
    ax.plot(x, dm_act_mu, lw=1.8, label="Actual biomass (DM)")
    ax.fill_between(x, dm_act_lo, dm_act_hi, alpha=0.15)
    ax.set_title("A) Biomass production (season-day aligned)", fontsize=12)
    ax.set_ylabel("g DM m$^{-2}$ day$^{-1}$")
    ax.legend()

        # -------------------------
    # Panel B (improved colors)
    # -------------------------
    ax = axes[0, 1]

    # Color choices (high contrast, readable)
    c_ftsw = "#1f4e79"   # dark blue
    c_irr  = "#b35a00"   # dark orange/brown
    c_thr  = "#444444"   # dark gray

    # FTSW (mean + CI)
    ax.plot(x, ftsw_mu, lw=2.2, color=c_ftsw, label="FTSW", zorder=3)
    ax.fill_between(x, ftsw_lo, ftsw_hi, color=c_ftsw, alpha=0.12, zorder=2)

    # Trigger line
    ax.axhline(params["Irrigation_Threshold"], color=c_thr, ls="--", lw=1.6,
               label="Irrigation trigger", zorder=4)

    ax.set_ylabel("FTSW (-)", color=c_ftsw)
    ax.tick_params(axis="y", colors=c_ftsw)
    ax.set_title("B) Soil water status + seasonal irrigation", fontsize=12)

    # Cumulative irrigation (right axis, mean + CI)
    ax2 = ax.twinx()
    ax2.plot(x, irr_mu, lw=2.2, color=c_irr, label="Cumulative irrigation", zorder=3)
    ax2.fill_between(x, irr_lo, irr_hi, color=c_irr, alpha=0.10, zorder=1)
    ax2.set_ylabel("Cumulative irrigation (mm)", color=c_irr)
    ax2.tick_params(axis="y", colors=c_irr)

    # Two separate legends (avoids mixing twin-axis handles)
    ax.legend(loc="upper left", frameon=True)
    ax2.legend(loc="upper right", frameon=True)


    ax = axes[1, 0]
    ax.plot(x, yld_mu, lw=2.0)
    ax.fill_between(x, yld_lo, yld_hi, alpha=0.15)
    ax.set_title("C) Yield accumulation (season-day aligned)", fontsize=12)
    ax.set_ylabel("g DM m$^{-2}$")

    ax = axes[1, 1]
    ax.plot(x, lai_mu, lw=2.0)
    ax.fill_between(x, lai_lo, lai_hi, alpha=0.15)
    ax.set_title("D) Canopy development (LAI)", fontsize=12)
    ax.set_ylabel("LAI (-)")

    plt.tight_layout()
    if save_figs:
        fig1.savefig(fig1_path, dpi=200, bbox_inches="tight")
        print(f"[INFO] Saved: {fig1_path}")
    plt.show()

    # Figure 2
    fig2, axes2 = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
    # fig2.suptitle("Figure 2 (Multi-season): Environmental & Physiological Drivers (Mean ± 95% CI)",
    #               fontsize=16, fontweight="bold")

    ax = axes2[0]
    ax.plot(x, tmin_mu, lw=1.6, label="Tmin")
    ax.fill_between(x, tmin_lo, tmin_hi, alpha=0.15)
    ax.plot(x, tmax_mu, lw=1.6, label="Tmax")
    ax.fill_between(x, tmax_lo, tmax_hi, alpha=0.15)
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("A) Thermal regime", fontsize=12)
    ax.legend(loc="upper right")

    ax = axes2[1]
    ax.plot(x, vpd_mu, lw=1.6, label="VPD (kPa)")
    ax.fill_between(x, vpd_lo, vpd_hi, alpha=0.15)
    ax.set_ylabel("VPD (kPa)")
    ax2 = ax.twinx()
    ax2.plot(x, tr_mu, lw=1.6, label="Transpiration (proxy)")
    ax2.fill_between(x, tr_lo, tr_hi, alpha=0.15)
    ax2.set_ylabel("Transpiration (mm/day, proxy)")
    ax.set_title("B) Atmospheric demand vs plant water use", fontsize=12)
    h1, l1 = ax.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax.legend(h1+h2, l1+l2, loc="upper left")

    ax = axes2[2]
    ax.plot(x, reltr_mu, lw=1.8, label="Relative transpiration (0–1)")
    ax.fill_between(x, reltr_lo, reltr_hi, alpha=0.15)
    ax.set_ylabel("Relative transpiration (-)")
    ax.set_title("C) Seasonal stress signature (1=unstressed)", fontsize=12)
    ax.legend(loc="upper right")

    plt.tight_layout()
    if save_figs:
        fig2.savefig(fig2_path, dpi=200, bbox_inches="tight")
        print(f"[INFO] Saved: {fig2_path}")
    plt.show()

plot_multiseason_ci(df, res_df, windows, params, save_figs=SAVE_FIGS)


[INFO] Radiation units likely MJ/m²/day; keeping (×1.0).
[INFO] Mean ALLSKY=24.71 | Mean SRAD_MJ=24.71 | Mean PAR_MJ=11.86

--- STARTING SIMULATION:  ---
Date window: 2024-08-31 to 2025-02-26
Irrigation trigger: FTSW < 0.20
Harvest DOY: 120 (calibrate if needed)

--- SEASON 2023-2024 RESET (Sep 1) ---
    HI_pot=0.350 | PClf_pot=0.180
[PHENO] 2024-09-08 dormancy released
[PHENO] 2024-10-04 anthesis reached
[HI] 2024-10-10 HIa fixed | HIa=0.350

[INFO] Complete seasons (Sep->May): []


KeyError: 'Yield_DM_kg_ha'